# Подгрузка пакетов и данных

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


In [28]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

from gensim.models.word2vec import Word2Vec

from shared.consts import FUNCTIONS_JSON_PROPERTIES_NAMES, PATH_TO_FUNCTIONS_STORAGE_JSON, PATH_TO_WORD2VEC_MODEL

Загрузим уже построенную модель Word2Vec

In [3]:
word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [4]:
functions_storage_dictionary = json.load(open(PATH_TO_FUNCTIONS_STORAGE_JSON))

# Отделение объявления функций от их использования

Функции с одним и тем же названием могут быть объявлены по-разному, поэтому задания функций будут склдываться в словарь по принципу:

Имя репозитория + имя функции

In [5]:
SEPARATOR_REPO_FUNCNAME = '___'

def get_key_in_func_creation_dict(func_name, file_path):
    paths_segments = file_path.split('/')
    
    if (len(paths_segments) > 2):
        return paths_segments[1] + SEPARATOR_REPO_FUNCNAME + func_name
    else:
        return func_name
    
def get_func_name_from_key(key_in_func_creation_dict):
    return key_in_func_creation_dict.split(SEPARATOR_REPO_FUNCNAME)[-1]

In [6]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionDeclaration' or
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionExpression'
               ):
                func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
                func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
                if func_creation_dict_key in func_creation_dic:
                    pass
                    #print(func_creation_dict_key, 'уже есть в словаре')
                func_creation_dic[func_creation_dict_key] = {
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES}" : function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES],
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH}": filePath
                }
    
    return func_creation_dic

In [7]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [8]:
def find_all_func_executions(func_storage_dic, func_creation_dic):
    func_exec_storage = []
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
            func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
            
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                func_creation_dict_key in func_creation_dic # должна быть в нашем словаре
            ):
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH] = filePath
                func_exec_storage.append(function)
    
    return func_exec_storage

In [9]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [10]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Размер словаря с объявлениями функций:  18626
Размер массива с вызовами функций:  2730


Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [11]:
func_creation_dic[get_key_in_func_creation_dict('confirmDelete', 'scripts/Block8/PHPCI/public/assets/js/phpci.js')]

{'argumentsNames': ['url', 'subject', 'reloadAfter'],
 'filePath': 'scripts/Block8/PHPCI/public/assets/js/phpci.js'}

Массив с вызовами функций состоит из словарей следующего формата

In [12]:
list(filter(lambda func_exec: func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] == 'confirmDelete',func_exec_storage))

[]

# Алгоритм проверки схожести аргументов

In [13]:
def checkIfWordInWord2VecDict(word, word2VecVocabulary = word2VecModel.wv):
    return word in word2VecVocabulary

def get_args_similarity(exec_func_name, exec_func_args_names, func_path, func_dict, word2VecModel):
    """Определяет схожесть названий аргументов в вызове функции с названиями аргументов в объявлении функции"""
    args_similarity = []
    
    func_in_func_creation_dic_key = get_key_in_func_creation_dict(exec_func_name, func_path)
    
    for argIndex in range(len(exec_func_args_names)):
        
        if argIndex > len(func_dict[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) - 1:
            args_similarity.append(None)
            continue
        
        exec_arg = exec_func_args_names[argIndex]
        create_arg = func_dict[get_key_in_func_creation_dict(exec_func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES][argIndex]
        
        if (checkIfWordInWord2VecDict(exec_arg) and checkIfWordInWord2VecDict(create_arg)):
            args_similarity.append(word2VecModel.wv.similarity(exec_arg, create_arg))
        else:
            args_similarity.append(None)
        
    return args_similarity

Посмотрим, как это работает на примере

In [14]:
example_func_exec = list(filter(lambda func: func[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME].endswith('updateLine'), func_exec_storage))
example_func_exec = example_func_exec[0]

func_name = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
func_path = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
args_names = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]

print('Вызов функции:', func_name, 'c аргументами', args_names)
func_in_func_creation_dic_key = get_key_in_func_creation_dict(func_name, example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH])
print('Функция же была объявлена с словаре с аргументами:', func_creation_dic[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])

get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel)

Вызов функции: updateLine c аргументами ['lastLine', 'lastSpans', 'estimateHeight']
Функция же была объявлена с словаре с аргументами: ['line', 'text', 'markedSpans', 'estimateHeight']


[0.44814897, -0.022563308, 0.33035174]

# Реализация на всем массиве вызовов функции

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Threshold

In [15]:
MIN_SIMILARITY_THRESHOLD = 0.1

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [16]:
for func_exec in func_exec_storage:
    func_name = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
    args_names = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]
    func_path = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
    
    args_similarities = get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel)
    is_below_threshold_args_similarities = list(map(lambda x: x and x < MIN_SIMILARITY_THRESHOLD, args_similarities))
    
    if (any(is_below_threshold_args_similarities)):
        print('В файле', func_path, 'проверь:')
        print('Была вызвана функция', func_name, 'с аргументами', args_names)
        print('Но вот объявлена функция была со следующими аргументами:')
        print(func_creation_dic[get_key_in_func_creation_dict(func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])
        print(args_similarities)
        print('_____________')

В файле scripts/Benvie/Node.js-Ultra-REPL/lib/utility/introspect.js проверь:
Была вызвана функция define с аргументами ['Descriptor']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
[-0.0072161336]
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['BitfieldT']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
[-0.018067012]
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['PointerT']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
[0.0016023864]
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['PointerT', 'PointeeType']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
[0.0016023864, 0.050306745]
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen

В файле scripts/bolinfest/plovr/closure/closure-library/closure/goog/testing/loosemock_test.js проверь:
Была вызвана функция assertContains с аргументами ['message']
Но вот объявлена функция была со следующими аргументами:
['boxB']
[0.09585209]
_____________
В файле scripts/bolinfest/plovr/closure/closure-library/closure/goog/testing/loosemock_test.js проверь:
Была вызвана функция assertContains с аргументами ['message']
Но вот объявлена функция была со следующими аргументами:
['boxB']
[0.09585209]
_____________
В файле scripts/bolinfest/plovr/closure/closure-library/closure/goog/ui/container_test.js проверь:
Была вызвана функция add с аргументами ['containerElement']
Но вот объявлена функция была со следующими аргументами:
['var_args']
[0.029137393]
_____________
В файле scripts/bolinfest/plovr/closure/closure-library/closure/goog/ui/editor/abstractdialog_test.js проверь:
Была вызвана функция assertContains с аргументами ['html']
Но вот объявлена функция была со следующими аргументами

В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _add_p_edges с аргументами ['$8', '$mp']
Но вот объявлена функция была со следующими аргументами:
['$chans', '$mp']
[0.09271349, 1.0]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _assignTrackNo с аргументами ['$9']
Но вот объявлена функция была со следующими аргументами:
['$chans']
[0.08723231]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _assignTrackNo с аргументами ['$10']
Но вот объявлена функция была со следующими аргументами:
['$chans']
[0.074166454]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _bezier_clip с аргументами ['$ictxt', '$23', '$3']
Но вот объявлена функция была со следующими аргументами:
['$inside_context', '$inside', '$sp', '$left_inside']
[0.072585575, None, -0.03836176]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js пров

В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция ___cxa_throw с аргументами ['$1', '__ZTISt9bad_alloc']
Но вот объявлена функция была со следующими аргументами:
['ptr', 'type', 'destructor']
[0.13990252, -0.07899723]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _free с аргументами ['start']
Но вот объявлена функция была со следующими аргументами:
['$mem']
[-0.054524142]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _free с аргументами ['temp']
Но вот объявлена функция была со следующими аргументами:
['$mem']
[0.012343196]
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _memcpy с аргументами ['newStr', 'ptr', 'len']
Но вот объявлена функция была со следующими аргументами:
['dest', 'src', 'num', 'align']
[0.32433617, 0.08985565, 0.31812453]
_____________
В файле scripts/calzoneman/sync/www/js/util.js проверь:
Был

В файле scripts/cdnjs/cdnjs/ajax/libs/immutable/2.3.2/Immutable.js проверь:
Была вызвана функция createClass с аргументами ['BitmapIndexedNode']
Но вот объявлена функция была со следующими аргументами:
['ctor', 'methods', 'staticMethods', 'superClass']
[0.01723604]
_____________
В файле scripts/cdnjs/cdnjs/ajax/libs/immutable/2.3.2/Immutable.js проверь:
Была вызвана функция createClass с аргументами ['HashCollisionNode']
Но вот объявлена функция была со следующими аргументами:
['ctor', 'methods', 'staticMethods', 'superClass']
[-0.022938725]
_____________
В файле scripts/cdnjs/cdnjs/ajax/libs/immutable/2.3.2/Immutable.js проверь:
Была вызвана функция createClass с аргументами ['MapIterator', 'Iterator']
Но вот объявлена функция была со следующими аргументами:
['ctor', 'methods', 'staticMethods', 'superClass']
[None, 0.062812805]
_____________
В файле scripts/cdnjs/cdnjs/ajax/libs/immutable/2.3.2/Immutable.js проверь:
Была вызвана функция createClass с аргументами ['VectorIterator', 'It

В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN12b2BroadPhase11UpdatePairsI16b2ContactManagerEEvPT_ с аргументами ['$3', '$2']
Но вот объявлена функция была со следующими аргументами:
['$this', '$callback']
[0.20328227, -0.01325422]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN5b2Rot3SetEf с аргументами ['$6', '$9']
Но вот объявлена функция была со следующими аргументами:
['$this', '$angle']
[0.23699372, 0.065077044]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN7b2Sweep7AdvanceEf с аргументами ['$6', '$7']
Но вот объявлена функция была со следующими аргументами:
['$this', '$alpha']
[0.23699372, 0.029272329]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN5b2Rot3SetEf с аргументами ['$20', '$23']
Но вот объявле

В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN7b2Joint11SetUserDataEPv с аргументами ['$3', '$4']
Но вот объявлена функция была со следующими аргументами:
['$this', '$data']
[0.20328227, -0.057267908]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN11b2GearJoint8SetRatioEf с аргументами ['$3', '$4']
Но вот объявлена функция была со следующими аргументами:
['$this', '$ratio']
[0.20328227, -0.07245576]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN7b2Joint11SetUserDataEPv с аргументами ['$4', '$5']
Но вот объявлена функция была со следующими аргументами:
['$this', '$data']
[0.2190086, -0.039258428]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN11b2WeldJoint12SetFrequencyEf с аргументами ['$3', '$4']
Но вот объявл

В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN6b2Body8SetAwakeEb с аргументами ['$4', '$6']
Но вот объявлена функция была со следующими аргументами:
['$this', '$flag']
[0.2190086, -0.03245585]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN6b2Body9SetBulletEb с аргументами ['$4', '$6']
Но вот объявлена функция была со следующими аргументами:
['$this', '$flag']
[0.2190086, -0.03245585]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN6b2Body16SetFixedRotationEb с аргументами ['$4', '$6']
Но вот объявлена функция была со следующими аргументами:
['$this', '$flag']
[0.2190086, -0.03245585]
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN6b2Body19ApplyAngularImpulseEf с аргументами ['$3', '$4']
Но вот объявлена функция 

## Эксперименты с моделями

### Трансформируем данные

Оставим только функции, у которых 2 аргумента

In [17]:
func_creation_dic_2_args = dict(
    filter(lambda elem: len(elem[1][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) == 2, func_creation_dic.items())
)

In [18]:
class TWO_ARGS_FUNC_COLUMNS:
    FIRST_ARG = 'arg1'
    SECOND_ARG = 'arg2'
    FUNC_NAME = 'funcName'
    TARGET = 'isBuggy'  # В верном ли порядке расставлены аргументы в функции

In [19]:
two_args_funcs = pd.DataFrame.from_dict(func_creation_dic_2_args, orient='index')
two_args_funcs[[TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG]] = pd.DataFrame(
    two_args_funcs.argumentsNames.tolist(), index=two_args_funcs.index
)

two_args_funcs[TWO_ARGS_FUNC_COLUMNS.FUNC_NAME] = list(
    map(lambda rowName: get_func_name_from_key(rowName), two_args_funcs.index)
)
two_args_funcs.drop(['argumentsNames', 'filePath'], axis='columns', inplace=True)
two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET] = 0

two_args_funcs.reset_index(inplace=True, drop=True)

print('Количество строк:', len(two_args_funcs))
two_args_funcs.head(3)

Количество строк: 3701


,arg1,arg2,funcName,isBuggy
0,stringerName,alertData,onAlert,0
1,child,parent,extend,0
2,el,highlighter,doHighlight,0


Меняем местами аргументами и добавляем эти строчки как isBuggy = 1

(то есть получим 50% одного класса и 50% другого класса)

In [20]:
buggy_code_rows = []

for rowIndex in tqdm(range(len(two_args_funcs))):
    new_row = {
        TWO_ARGS_FUNC_COLUMNS.FUNC_NAME: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
        TWO_ARGS_FUNC_COLUMNS.FIRST_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG],
        TWO_ARGS_FUNC_COLUMNS.SECOND_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG],
        TWO_ARGS_FUNC_COLUMNS.TARGET: 1
    }
    buggy_code_rows.append(new_row)

two_args_funcs = two_args_funcs.append(pd.DataFrame(buggy_code_rows)).reset_index(drop=True)

In [21]:
print('Количество строк:', len(two_args_funcs))
two_args_funcs.tail(3)

Количество строк: 7402


,arg1,arg2,funcName,isBuggy
7399,metric,paramGroup,getParamValueFromPath,1
7400,metric,paramGroup,applyRegexToName,1
7401,data,target,renderExtendedGraph,1


Теперь заменяем слова на их векторное представление из word2Vec.

Если например, каждое слово представлено в модели word2Vec как вектор из 300 значений, то название функции + 2 аргумента + 1 целевая перееменная (isBuggy) = 901 переменная в датасете

In [22]:
def replace_string_column_with_vectors(pd_df, columnName, word2VecVocabulary = word2VecModel.wv):
    DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC = '0'
    
    vector_size = len(word2VecVocabulary[DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC])
    
    future_columns_names = []
    
    vectorized_df = pd.DataFrame()
    
    for i in range(vector_size):
        future_columns_names.append(columnName + str(i))
    
    for rowIndex in tqdm(range(len(pd_df))):
        word = pd_df.loc[rowIndex, columnName]
        
        if not checkIfWordInWord2VecDict(word, word2VecVocabulary):
            word = DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC
        
        newColumnsDf = pd.DataFrame(
            word2VecVocabulary[word].reshape(-1, len(word2VecVocabulary[word])), columns = future_columns_names
        )
        vectorized_df = vectorized_df.append(newColumnsDf)
    
    return pd.concat([pd_df, vectorized_df.reset_index(drop=True)], axis=1)

In [23]:
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

two_args_funcs = two_args_funcs.drop(
    [TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
    axis=1
)

In [24]:
two_args_funcs.head(5)

,isBuggy,arg10,arg11,arg12,arg13,arg14,arg15,arg16,arg17,arg18,...,funcName290,funcName291,funcName292,funcName293,funcName294,funcName295,funcName296,funcName297,funcName298,funcName299
0,0,0.008887,-1.366884,0.379557,-0.637870,-0.983469,-0.799238,1.835120,0.867309,0.507156,...,-1.706656,-0.016910,-0.163960,0.307146,0.340781,-0.584781,1.014504,0.285870,-0.039231,-0.678411
1,0,1.821814,-0.064067,0.530646,-0.654173,1.204931,-1.622632,-0.982288,-0.744273,-0.207713,...,1.774638,-0.324511,0.326671,-0.007815,-0.505875,-0.176359,2.011366,0.732395,2.775929,-3.347799
2,0,-0.538299,0.394476,-0.210604,-0.989659,-0.242101,-0.445696,-0.897229,0.258255,-0.315091,...,0.274681,0.130173,0.011621,0.283104,0.010377,0.026425,-0.060031,0.156571,0.064422,-0.215059
3,0,-1.300357,1.030672,1.287416,-0.271649,-0.630023,0.204095,0.460159,1.023795,1.531788,...,-1.706656,-0.016910,-0.163960,0.307146,0.340781,-0.584781,1.014504,0.285870,-0.039231,-0.678411
4,0,0.004450,0.035352,0.070651,-0.026263,0.077220,0.044717,-0.094510,0.032139,-0.015525,...,-1.706656,-0.016910,-0.163960,0.307146,0.340781,-0.584781,1.014504,0.285870,-0.039231,-0.678411


### Обучаем

Кормим это дело модели

In [25]:
y = np.array(two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET])

X = two_args_funcs.drop(TWO_ARGS_FUNC_COLUMNS.TARGET, axis = 1)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rf = RandomForestClassifier(n_estimators = 1000, random_state = 27)

rf.fit(X, y);

### Проверяем на обучающей

In [33]:
y_train_pred = rf.predict(X)
print('accuracy:', metrics.accuracy_score(y, y_train_pred))
print('recall:', metrics.recall_score(y, y_train_pred))
print('precision:', metrics.precision_score(y, y_train_pred))

accuracy: 0.9812212915428262
recall: 0.9886517157524993
precision: 0.974174653887114


### Проверяем на тестовой выборке

Формируем тестовую выборку из массива с вызовами функциями

Вновь фильтруем только функции с 2 аргументами

In [27]:
func_exec_storage

[{'type': 'CallExpression',
  'argumentsNames': ['trigger'],
  'functionName': 'addTrigger',
  'filePath': 'scripts/BayardRock/WebIntellisense/src/js/lib/webintellisense-codemirror.js'},
 {'type': 'CallExpression',
  'argumentsNames': ['trigger'],
  'functionName': 'addTrigger',
  'filePath': 'scripts/BayardRock/WebIntellisense/src/js/lib/webintellisense-codemirror.js'},
 {'type': 'CallExpression',
  'argumentsNames': ['editor'],
  'functionName': 'setEditor',
  'filePath': 'scripts/BayardRock/WebIntellisense/src/js/lib/webintellisense-codemirror.js'},
 {'type': 'CallExpression',
  'argumentsNames': ['func'],
  'functionName': 'runs',
  'filePath': 'scripts/BenjaminAdams/RedditJS/public/js/lib/jasmine.js'},
 {'type': 'CallExpression',
  'argumentsNames': ['timeout'],
  'functionName': 'waits',
  'filePath': 'scripts/BenjaminAdams/RedditJS/public/js/lib/jasmine.js'},
 {'type': 'CallExpression',
  'argumentsNames': ['beforeEachFunction'],
  'functionName': 'beforeEach',
  'filePath': 'sc

превращаем в датафрейм

векторизуем перемененные 

In [ ]:
test_df = replace_string_column_with_vectors(test_df, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
test_df = replace_string_column_with_vectors(test_df, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
test_df = replace_string_column_with_vectors(test_df, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

two_args_funcs = two_args_funcs.drop(
    [TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
    axis=1
)